## Visualizaing data

### Building a Google map from geocoded data

In [11]:
!pwd

/home/swchen/github/py4e


In [4]:
!cd geodata

In [9]:
!cd geodata

In [12]:
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'

if api_key is False:
    api_key = 42
    serviceurl = "http://py4e-data.dr-chuck.net/json?"
else :
    serviceurl = "https://maps.googleapis.com/maps/api/geocode/json?"

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open("geodata/where.data")
count = 0
for line in fh:
    if count > 200 :
        print('Retrieved 200 locations, restart to retrieve more')
        break

    address = line.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        data = cur.fetchone()[0]
        print("Found in database ",address)
        continue
    except:
        pass

    parms = dict()
    parms["address"] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') :
        print('==== Failure To Retrieve ====')
        print(data)
        break

    cur.execute('''INSERT INTO Locations (address, geodata)
            VALUES ( ?, ? )''', (memoryview(address.encode()), memoryview(data.encode()) ) )
    conn.commit()
    if count % 10 == 0 :
        print('Pausing for a bit...')
        time.sleep(5)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")


Retrieving http://py4e-data.dr-chuck.net/json?address=AGH+University+of+Science+and+Technology&key=42
Retrieved 2315 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=Academy+of+Fine+Arts+Warsaw+Poland&key=42
Retrieved 1435 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=American+University+in+Cairo&key=42
Retrieved 1246 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=Arizona+State+University&key=42
Retrieved 1802 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=Athens+Information+Technology&key=42
Retrieved 1753 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=BITS+Pilani&key=42
Retrieved 1645 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json?address=Babcock+University&key=42
Retrieved 1635 characters {    "results" : [  

Retrieving http://py4e-data.dr-chuck.net/json

### geodump.py

In [13]:
import sqlite3
import json
import codecs

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')
fhand = codecs.open('where.js', 'w', "utf-8")
fhand.write("myData = [\n")
count = 0
for row in cur :
    data = str(row[1].decode())
    try: js = json.loads(str(data))
    except: continue

    if not('status' in js and js['status'] == 'OK') : continue

    lat = js["results"][0]["geometry"]["location"]["lat"]
    lng = js["results"][0]["geometry"]["location"]["lng"]
    if lat == 0 or lng == 0 : continue
    where = js['results'][0]['formatted_address']
    where = where.replace("'", "")
    try :
        print(where, lat, lng)

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")
cur.close()
fhand.close()
print(count, "records written to where.js")
print("Open where.html to view the data in a browser")

aleja Adama Mickiewicza 30, 30-059 Kraków, Poland 50.06688579999999 19.9136192
Krakowskie Przedmieście 5, 00-068 Warszawa, Poland 52.2394019 21.0150792
AUC Avenue، 11835, Egypt 30.018923 31.499674
Tempe, AZ 85281, USA 33.4242399 -111.9280527
Monumental Plaza, Building C, 1st Floor, Leof. Kifisias 44, Marousi 151 25, Greece 38.0399391 23.8030901
VidyaVihar Campus, street number 41, Pilani, Rajasthan 333031, India 28.3639976 75.58696809999999
Ilishan-Remo, Nigeria 6.8946472 3.7174267
Ajagara, Varanasi, Uttar Pradesh 221005, India 25.2677203 82.99125819999999
Gnana Bharathi Campus, Gnana Bharathi Main Rd, Teachers Colony, Nagarbhavi, Bengaluru, Karnataka 560056, India 12.9527314 77.5157387
1301 S University Parks Dr, Waco, TX 76706, USA 31.5497007 -97.1143046
19 Xinjiekou Outer St, BeiTaiPingZhuang, Haidian Qu, Beijing Shi, China, 100875 39.9619537 116.3662615
Praspyekt Nyezalyezhnastsi 4, Minsk, Belarus 53.8938988 27.5460609
Studentski trg 1, Beograd, Serbia 44.8184339 20.4575676
700 Col